## 简答题

1. TensorFlow 是否可以简单替代 NumPy？两者之间的主要区别是什么？
- 可以，numpy多數函數都有對應的tensorflow函數，兩者間的主要區別在於數據類型與定義方式不同，但在複雜編程中則需要慎重考慮，因為兩者有不同的運行規則及適用場景；
- Numpy多沿用python原本數據結構做為底層邏輯(多維數組)，而TF使用了張量與變量作為基礎數據


2. 使用 `tf.range(10)` 和 `tf.constant(np.arange(10))` 是否会得到相同的结果？
- 不相同，tf.range默认创建的数据类型为32位浮点数，而np.arange(10)使用64位浮点数


3. 可以通过编写函数或继承 `tf.keras.losses.Loss` 来定义自定义损失函数。两种方法分别应该在什么时候使用？
- 继承自tf.keras.losses.Loss的自定义类式损失函数被用于随着模型保存与恢复超参数，如果没有这种需求可以通过函数式实现，这样更简单


4. 可以直接在函数中定义自定义指标或采用 `tf.keras.metrics.Metric` 子类。两种方法分别应该在什么时候使用？
- 针对于简单指标可以写成函数式(简易)，针对复杂指标则需要超类继承的自定义类式


5. 什么时候应该自定义层而不是自定义模型？
- 当tf.keras内置的层无法满足需求时，或是网络架构中存在许多重复结构为了更清晰的编程表达，可以考虑自定义层


6. 有哪些示例需要编写自定义训练循环？
- 典型例子有宽深神经网络论文中展示的内容：因为其宽路径与深路径各使用了不同的优化器，需要自行编写自定义训练循环以支持这一点


7. 自定义 Keras 组件中可以包含任意 Python 代码，还是必须转换为 TF 函数？
- 尽量转换为TF函数以确保兼容和可移植性，像是通过@tf.function修饰器等，以避免不可预见的问题破坏可移植性

8. 如果要将函数转换为 TF 函数，应避免哪些主要模式？
- 避免使用原生循环，不同的Py函数必须遵守相同规则，随机数生成必须使用TF模块提供的方法，并需要注意带有额外执行的py函数可能不会每一次都执行

9. 何时需要创建动态 Keras 模型？ 如何动态创建Keras模型？为什么不是所有模型都动态化？
- 模型结构于前向传播中依赖输入数据，或需要灵活控制执行流程时会需要动态模型
- 通过继承tf.keras.Model的自定义类并重写call()方法实现
- 相比静态模型训练效率较低、更难达成保存与加载等序列操作，且无法像计算图那般进行全面优化


In [4]:
import numpy as np
import tensorflow as tf
from keras.activations import softmax

tf.constant(np.arange(10)) # <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>
tf.range(10) # <tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

## 编程题

1. 实现一个执行层归一化的自定义层：
    - a. `build()` 方法应定义两个可训练的权重 α 和 β，它们的形状均为 `input_shape[-1:]`，数据类型为 `tf.float32`。α 应该用 1 初始化，而 β 必须用 0 初始化。
    - b. `call()` 方法应计算每个实例特征的均值和标准差。为此，可以使用 `tf.nn.moments(inputs, axes=-1, keepdims=True)`，它返回同一实例的均值 μ 和方差 σ²（计算方差的平方根便可获得标准差）。然后，该函数应计算并返回
      $$
      \alpha \otimes \frac{(X-\mu)}{(\sigma+\epsilon)} + \beta
      $$
      其中 ε 是表示项精度的一个常量（避免被零除的小常数，例如 0.001）,$\otimes$表示逐个元素相乘
    - c. 确保自定义层产生与tf.keras.layers.LayerNormalization层相同（或几乎相同）的输出。

2. 使用自定义训练循环训练模型来处理Fashion MNIST数据集（13_神经网络介绍 里用的数据集）：

    - a.显示每个轮次、迭代、平均训练损失和每个轮次的平均精度（在每次迭代中更新），以及每个轮次结束时的验证损失和精度。
    - b.尝试对上面的层和下面的层使用具有不同学习率的不同优化器。

In [5]:
# 1. 实现一个执行层归一化的自定义层：
#     - a. `build()` 方法应定义两个可训练的权重 α 和 β，它们的形状均为 `input_shape[-1:]`，数据类型为 `tf.float32`。α 应该用 1 初始化，而 β 必须用 0 初始化。
#     - b. `call()` 方法应计算每个实例特征的均值和标准差。为此，可以使用 `tf.nn.moments(inputs, axes=-1, keepdims=True)`，它返回同一实例的均值 μ 和方差 σ²（计算方差的平方根便可获得标准差）。然后，该函数应计算并返回
#       $$
#       \alpha \otimes \frac{(X-\mu)}{(\sigma+\epsilon)} + \beta
#       $$
#       其中 ε 是表示项精度的一个常量（避免被零除的小常数，例如 0.001）,$\otimes$表示逐个元素相乘
#     - c. 确保自定义层产生与tf.keras.layers.LayerNormalization层相同（或几乎相同）的输出。

import tensorflow as tf
class MyLayerNormalization(tf.keras.layers.Layer):
    def __init__(self, axis=-1, epsilon=1e-3, **kwargs):
        # axis: 要归一化的轴（默认为 -1，即最后一个轴）
        # epsilon: 避免除以 0 的小常数（题目示例用了 0.001）
        super().__init__(**kwargs)
        if isinstance(axis, int):
            self.axis = (axis,)
        else:
            self.axis = tuple(axis)
        self.epsilon = float(epsilon)

    def build(self, input_shape):
        # input_shape 是一个序列（例如 (batch, ..., dim)）
        input_shape = tf.TensorShape(input_shape).as_list()
        # 取最后一维大小作为参数的形状
        dim = int(input_shape[-1])
        # α 初始化为 1，β 初始化为 0；dtype 要求题目是 tf.float32
        self.alpha = self.add_weight(
            name="alpha",
            shape=(dim,),
            initializer=tf.keras.initializers.Ones(),
            dtype=tf.float32,
            trainable=True
        )
        self.beta = self.add_weight(
            name="beta",
            shape=(dim,),
            initializer=tf.keras.initializers.Zeros(),
            dtype=tf.float32,
            trainable=True
        )
        super().build(input_shape)

    def call(self, inputs, training=None):
        # 1) 计算按最后一维（features）每个实例的均值和方差
        #    keepdims=True 保持最后一维为 1，便于广播
        mean, var = tf.nn.moments(inputs, axes=self.axis, keepdims=True)
        std = tf.sqrt(var)

        # 2) 标准化： (X - μ) / (σ + ε)
        normalized = (inputs - mean) / (std + self.epsilon)

        # 3) 把 alpha/beta 转为和 inputs 相同的数据类型再广播
        alpha = tf.cast(self.alpha, inputs.dtype)
        beta = tf.cast(self.beta, inputs.dtype)

        # alpha 的形状是 (features,) —— 这会自动广播到 (..., features)
        return alpha * normalized + beta

    def get_config(self):
        cfg = super().get_config()
        cfg.update({"axis": self.axis, "epsilon": self.epsilon})
        return cfg


In [9]:
# 2. 使用自定义训练循环训练模型来处理Fashion MNIST数据集（13_神经网络介绍 里用的数据集）：
#
#     - a.显示每个轮次、迭代、平均训练损失和每个轮次的平均精度（在每次迭代中更新），以及每个轮次结束时的验证损失和精度。
#     - b.尝试对上面的层和下面的层使用具有不同学习率的不同优化器。
import tensorflow as tf

# 加载Fashion MNIST。它已经被打乱并分成了训练集（60000个图像）和测试集（10000个图像）
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test , y_test) = fashion_mnist

# 保留训练集中的最后5000个图像进行验证
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds


In [ ]:
# ========== 模型 ==========
class WideDeepModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.wide = tf.keras.layers.Dense(1)   # wide 部分
        self.hidden1 = tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal")
        self.hidden2 = tf.keras.layers.Dense(30, activation="relu", kernel_initializer="he_normal")
        self.out = tf.keras.layers.Dense(1)

    def call(self, inputs):
        wide_out = self.wide(inputs)
        deep_out = self.hidden1(inputs)
        deep_out = self.hidden2(deep_out)
        return self.out(wide_out + deep_out)   # wide + deep

model = WideDeepModel()

# ========== 优化器 ==========
wide_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
deep_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

loss_fn = tf.keras.losses.MeanSquaredError()
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_mae = tf.keras.metrics.MeanAbsoluteError(name="train_mae")
val_loss = tf.keras.metrics.Mean(name="val_loss")
val_mae = tf.keras.metrics.MeanAbsoluteError(name="val_mae")

# ========== 训练循环 ==========
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size

for epoch in range(1, n_epochs + 1):
    print(f"\nEpoch {epoch}/{n_epochs}")
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train, batch_size)
        with tf.GradientTape(persistent=True) as tape:
            y_pred = model(X_batch, training=True)
            loss_value = loss_fn(y_batch, y_pred)

        # 分别获取 wide 和 deep 参数
        wide_vars = model.wide.trainable_variables
        deep_vars = model.hidden1.trainable_variables + \
                    model.hidden2.trainable_variables + \
                    model.out.trainable_variables

        wide_grads = tape.gradient(loss_value, wide_vars)
        deep_grads = tape.gradient(loss_value, deep_vars)

        wide_optimizer.apply_gradients(zip(wide_grads, wide_vars))
        deep_optimizer.apply_gradients(zip(deep_grads, deep_vars))

        train_loss(loss_value)
        train_mae(y_batch, y_pred)

        # 进度条
        if step % 100 == 0 or step == n_steps:
            print(f"\rStep {step}/{n_steps} - "
                  f"loss: {train_loss.result():.4f} - "
                  f"mae: {train_mae.result():.4f}", end="")

    # 验证集评估
    y_val_pred = model(X_valid_scaled, training=False)
    val_loss(loss_fn(y_valid, y_val_pred))
    val_mae(y_valid, y_val_pred)

    print(f" - val_loss: {val_loss.result():.4f} - val_mae: {val_mae.result():.4f}")

    # reset metrics
    train_loss.reset_states()
    train_mae.reset_states()
    val_loss.reset_states()
    val_mae.reset_states()
